# データ分析入門編：scikit-learnで回帰！

今回は、有名はBoston House-Pricesデータセットを使い、scikit-learnで回帰モデルの作成します。
使うアルゴリズムはSVRとRandom Forestですが、他のアルゴリズムも同様に作成できます。
Boston House-Pricesデータセットはscikit-learnに組み込まれていますので、まずはデータセットをロードします。

In [1]:
import sklearn
from sklearn import datasets

boston = datasets.load_boston()

## bostonの型について
この"boston"は、Bunchというscikit-learn独自の辞書のような型です。

In [2]:
type(boston)

sklearn.utils.Bunch

## bostonの中身
まず、bostonのキーを見てみましょう。御覧の通り、次の５つが入っています。
1. DESCR: データの説明文
1. data：データ本体
1. target：予測したい変数のデータ
1. feature_names：dataの各列の名前

In [3]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

## Bostonの説明

それでは、まずは説明から見てみましょう。基本的な情報（何のデータなのか、データの数など）が記載されています。より詳しくはwikipediaなどを参照ください。

In [4]:
print(boston['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [5]:
boston['feature_names']

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [6]:
boston['data'][:10]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
        1.5300e+01, 3.9690e+02, 4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9283e+02, 4.0300e+00],
       [3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9463e+02, 2.9400e+00],
       [6.9050e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        7.1470e+00, 5.4200e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9690e+02, 5.3300e+00],
       [2.9850e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        6.4300e+00, 5.8700e

In [7]:
boston['target'][:10]

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9])

# SVRで回帰

SVRの場合は事前にデータセットを正規化する必要があります。scikit-learnにはこのようなプリプロセシング用のツールが多数あります。その中で、今回は平均０、標準偏差１にデータを正規化するStandardScalerを使います。

StandardScalerの詳細については[ここ](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)を参照ください。

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(boston.data)
data = scaler.transform(boston.data)

次に、学習用データセットと評価用データセットに分ける必要がありますが、scikit-learnにはそれを簡単にできるツールがあります。名前もtrain_test_splitです。再現性のためのrandom_stateを指定するのがおすすめです。

train_test_splitの詳細については[ここ](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)を参照ください。

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, boston.target, test_size=0.4, random_state=0)

scikit-learnの素晴らしいところはアルゴリズムが何であれ、その使い方が（基本的に）一緒だということ点です。
次の３行でモデルを作成できます。

```python
from sklearn.hoge import fuga
clf = fuga()
clf.fit(X_train, y_train)
```
尚、各アルゴリズムにはそれぞれ特有のパラメーターがありますが、それらは`clf=fuga()`の引数にパラメーターを指定します。
例えば、SVRの場合は次の通りです。

In [10]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(X_train, y_train) 

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

未知のデータに対して予測を行い場合はpredictメソッドを使います。例えば、今回学習に使っていないX_testで予測する場合は次の通りです。

In [11]:
clf.predict(X_test)

array([25.10579585, 19.01183768, 24.13978332, 13.93993634, 21.11022874,
       19.57890541, 20.7352087 , 20.25228083, 18.79412721, 18.68393209,
       20.90446191, 13.82165026, 14.69932714, 13.52103208, 26.33154875,
       30.38977904, 21.42269762, 31.59203198, 28.6802751 , 21.56537335,
       23.89571997, 21.59364973, 18.88558311, 28.21617568, 20.73675589,
       17.8543488 , 17.2821231 , 18.64699716, 33.18711461, 18.64752067,
       15.45819648, 17.28962857, 19.66241778, 21.81375679, 26.09024954,
       20.94118972, 12.7473424 , 22.25021711, 13.96926454, 13.24082191,
       23.82124823, 20.26656644, 23.01878395, 15.00546287, 25.05888937,
       23.56872953, 19.27765999, 21.21153828, 17.03167203, 23.03546781,
       20.78642411, 19.23793456, 21.5974788 , 28.62162607, 15.07353054,
       21.32779198, 20.6030634 , 18.32444129, 20.19770992, 21.52678564,
       21.10698864, 20.96138843, 31.55689366, 28.13075376, 18.53976672,
       30.26342019, 16.08656085, 20.24117289, 14.16229313, 21.98

In [12]:
y_test

array([22.6, 50. , 23. ,  8.3, 21.2, 19.9, 20.6, 18.7, 16.1, 18.6,  8.8,
       17.2, 14.9, 10.5, 50. , 29. , 23. , 33.3, 29.4, 21. , 23.8, 19.1,
       20.4, 29.1, 19.3, 23.1, 19.6, 19.4, 38.7, 18.7, 14.6, 20. , 20.5,
       20.1, 23.6, 16.8,  5.6, 50. , 14.5, 13.3, 23.9, 20. , 19.8, 13.8,
       16.5, 21.6, 20.3, 17. , 11.8, 27.5, 15.6, 23.1, 24.3, 42.8, 15.6,
       21.7, 17.1, 17.2, 15. , 21.7, 18.6, 21. , 33.1, 31.5, 20.1, 29.8,
       15.2, 15. , 27.5, 22.6, 20. , 21.4, 23.5, 31.2, 23.7,  7.4, 48.3,
       24.4, 22.6, 18.3, 23.3, 17.1, 27.9, 44.8, 50. , 23. , 21.4, 10.2,
       23.3, 23.2, 18.9, 13.4, 21.9, 24.8, 11.9, 24.3, 13.8, 24.7, 14.1,
       18.7, 28.1, 19.8, 26.7, 21.7, 22. , 22.9, 10.4, 21.9, 20.6, 26.4,
       41.3, 17.2, 27.1, 20.4, 16.5, 24.4,  8.4, 23. ,  9.7, 50. , 30.5,
       12.3, 19.4, 21.2, 20.3, 18.8, 33.4, 18.5, 19.6, 33.2, 13.1,  7.5,
       13.6, 17.4,  8.4, 35.4, 24. , 13.4, 26.2,  7.2, 13.1, 24.5, 37.2,
       25. , 24.1, 16.6, 32.9, 36.2, 11. ,  7.2, 22

尚、回帰問題の場合、モデルの評価指標として平均二乗誤差が使われます。
scikit-learnでは、mean_squared_errorという関数がありますので、こちらで計算できます。

In [13]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, clf.predict(X_test))

34.165910124601005

# RandomForestRegressorで回帰

下記の通り、SVRであれ、RandomForestRegressorであれ、モデルの学習方法、分類方法が同じです。

In [14]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=100)
clf.fit(X_train, y_train) 
mean_squared_error(y_test, clf.predict(X_test))

13.83375885221675